In [ ]:
import os
import numpy as np
# import pandas as pd
import xarray as xr

from esdglider import gcp, glider, utils

deployment_name = "george-20240530"
mode = "delayed"


# Standard 
bucket_name = 'amlr-gliders-deployments-dev'
deployments_path = f"/home/sam_woodman_noaa_gov/{bucket_name}"
config_path = f"/home/sam_woodman_noaa_gov/glider-lab/deployment-configs"

gcp.gcs_mount_bucket("amlr-gliders-deployments-dev", deployments_path, ro=False)
deployment_info = {
    "deploymentyaml": os.path.join(config_path, f"{deployment_name}.yml"), 
    "mode": mode, 
}
paths = glider.get_path_deployment(deployment_info, deployments_path)

dir_ts = paths["tsdir"]
path_raw = os.path.join(dir_ts, f"{deployment_name}-{mode}-raw.nc")
path_sci = os.path.join(dir_ts, f"{deployment_name}-{mode}-sci.nc")
path_eng = os.path.join(dir_ts, f"{deployment_name}-{mode}-eng.nc")

## Raw dataset

In [ ]:
ds_raw = xr.load_dataset(path_raw)
df_raw = ds_raw.to_pandas()
ds_raw

In [ ]:
ds_depth = utils.check_depth(ds_raw["depth"], ds_raw["depth_ctd"])
display(ds_depth)

df_depth = ds_depth.to_pandas()
df_depth.sort_values(by="depth_diff_abs", ascending=False)

In [ ]:
# Get only depth difference > 10
df_curr = df_depth[df_depth.depth_diff > 10]
df_curr.sort_values(by="time", ascending=True)
df_curr

In [ ]:
# Explore specific chunks of dataset
ds = utils.data_var_reorder(ds_raw, ["depth", "depth_ctd"])
dt = "2024-01-01"
ds_sub = ds.sel(time=slice(f"{dt} 02:15", dt))
ds_sub

## Science/engineering

In [ ]:
ds_eng = xr.load_dataset(path_eng)
df_eng = ds_eng.to_pandas()
display(ds_eng)

ds_sci = xr.load_dataset(path_sci)
df_sci = ds_sci.to_pandas()
display(ds_sci)

Science/engineering datasets have been filtered for bogus values
Interpolate the engineering dataset onto the science timestamps, and do the same depth check

In [ ]:
ds_depth2 = utils.check_depth(ds_eng["depth"], ds_sci["depth"])
display(ds_depth2)

df_depth2 = ds_depth2.to_pandas()
df_depth2.sort_values(by="depth_diff_abs", ascending=False)

## SOCIB

In [ ]:
sl1 = xr.load_dataset("/home/sam_woodman_noaa_gov/depfrbyrd_amlr03_sgg3_L1_2023-11-27_data_rt.nc")
display(sl1)

In [ ]:
display(sl1)

In [ ]:
sl1["depth_diff"] = sl1.depth - sl1.depth_ctd
display(sl1.depth_diff.to_pandas().describe())